# **Aplicaciones relativistas: GPS**

## **Trabajo computacional #4**

**Integrantes:**

- Santiago Londoño Álvarez — CC 1000084281  
- Juan Pablo Sánchez Arroyave — CC 1000873471  

In [2]:
#@title Librerias

import math

## **Objetivo**

Estudiar la aplicación de la física relativista (tanto especial como general) en el funcionamiento de los sistemas GPS.

### ***Objetivos generales***

- Analizar la aproximación newtoniana con correcciones relativistas.  
- Discutir la aproximación puramente relativista (SYPOR project).  



## ***Aproximación newtoniana con correcciones relativistas***


### ***Funcionamiento del GPS y Correcciones Relativistas***

El Sistema de Posicionamiento Global (GPS) consta de una constelación de al menos 24 satélites en órbitas circulares a una altitud aproximada de 20,200 km sobre la superficie terrestre, con un período orbital de 12 horas sidéreas. Cada satélite emite señales de radio que incluyen su posición orbital y una marca de tiempo precisa, generada por relojes atómicos de cesio o rubidio a bordo. La posición del receptor en Tierra se determina mediante triangulación, midiendo el tiempo de viaje de las señales desde al menos cuatro satélites [1-2].

Sin embargo, para lograr precisiones del orden de metros, deben considerarse efectos relativistas derivados de la teoría de la relatividad especial y general de Einstein. Estos incluyen [3]:

- **Dilatación temporal cinemática** (efecto de segundo orden del Doppler relativista): Debida a la velocidad orbital de los satélites ($v \approx 3.87$ km/s), los relojes a bordo se retrasan respecto a los en reposo en Tierra.
- **Dilatación temporal gravitacional** (corrimiento gravitacional o redshift/blueshift): Debida al potencial gravitacional más débil en la órbita, los relojes satelitales avanzan respecto a los terrestres.
- **Efecto Sagnac**: Debido a la rotación de la Tierra, afecta la propagación de las señales.
- **Otros efectos menores**: Como la oblatura terrestre (término cuadrupolar J2) y la relatividad de la simultaneidad.

Sin estas correcciones, los errores acumulados en el tiempo llevarían a desviaciones posicionales de hasta decenas de km por día.



### ***Efecto Doppler Relativista en Señales Satelitales***

El efecto Doppler relativista en señales de satélites GNSS (como GPS) combina la dilatación temporal por velocidad (**relatividad especia**l) y el corrimiento al rojo gravitacional (**relatividad general**), ajustado por la transformación entre marcos de referencia inercial (**ECI**) y rotante (E**CEF**). Esta derivación asume una aproximación post-newtoniana de orden $1/c^2$, válida para $v \ll c$ y campos gravitacionales débiles (métrica de Schwarzschild aproximada) [1,4].

*Fórmula Básica en Marco Inercial (ECI):*

En el marco ECI (Earth-Centered Inertial), la frecuencia recibida $f$ de una señal emitida a frecuencia $f_e$ por un satélite en movimiento se modifica por:
- Doppler relativista: Factor $\gamma (1 - \mathbf{N} \cdot \mathbf{v}/c)$, donde $\gamma = 1/\sqrt{1 - v^2/c^2} \approx 1 + v^2/(2c^2)$ (aproximación de segundo orden).
- Redshift gravitacional: Factor $1 + \Delta \Phi / c^2$, donde $\Delta \Phi = \Phi_r - \Phi_s$ es la diferencia de potencial gravitacional entre receptor ($r$) y satélite ($s$).

La fórmula combinada exacta (para propagación en vacío) es:
$$
f = f_e \frac{\sqrt{1 - v^2/c^2}}{1 - \mathbf{N} \cdot \mathbf{v}/c} \left(1 + \frac{\Delta \Phi}{c^2}\right),
$$
donde $\mathbf{v}$ es la velocidad del satélite relativa al receptor, $\mathbf{N}$ el vector unitario en la dirección de propagación. Esta surge de la conservación de la fase a lo largo de geodésicas nulas en espacio-tiempo plano con perturbación gravitacional.

En aproximación de bajo orden ($v \ll c$, $\Delta \Phi \ll c^2$):
$$
\frac{f}{f_e} \approx 1 - \frac{\mathbf{N} \cdot \mathbf{v}}{c} + \frac{v^2}{2c^2} + \frac{\Delta \Phi}{c^2}.
$$


*Transformación a Marco Rotante (ECEF):*


El marco ECEF rota con la Tierra a velocidad angular $\boldsymbol{\omega}_E \approx 7.2921151467 \times 10^{-5}$ rad/s alrededor del eje z. Las coordenadas se transforman vía matriz de rotación $R^S(t)$:
$$
\mathbf{X}^{\text{ECEF}} = R^S(t) \mathbf{X}^{\text{ECI}},
$$
con
$$
R^S(t) = \begin{pmatrix}
\cos \Theta(t) & \sin \Theta(t) & 0 \\
-\sin \Theta(t) & \cos \Theta(t) & 0 \\
0 & 0 & 1
\end{pmatrix},
$$
donde $\Theta(t) = \omega_E (t - t_k)$ y $t_k$ es un instante de referencia (e.g., para alinear ejes en $t_k$). Esta matriz surge de la integración de la ecuación de Euler para rotación rígida uniforme.

La velocidad en ECEF incluye un término adicional por rotación: $\mathbf{v}^{\text{ECEF}} = \mathbf{v}^{\text{ECI}} + \boldsymbol{\omega}_E \times \mathbf{r}$.

In [17]:
#@title Código para el cálculo de la corrección por día

# Constantes
c = 299792458  # m/s
GM = 3.986004418e14  # m^3/s^2 (constante gravitacional Tierra, WGS84)
r_sat = 26560000  # m (radio orbital aproximado GPS, semisemieje mayor 26561.75 km)
r_earth = 6378137  # m (radio ecuatorial Tierra, WGS84)
t_day = 86400  # s (segundos en un día)

# Velocidad orbital del satélite (aproximación circular)
v = math.sqrt(GM / r_sat)  # m/s

# Efecto de velocidad (relatividad especial, segundo orden Doppler/time dilation)
delta_f_v = - (v**2) / (2 * c**2)  # fracción adimensional

# Efecto gravitacional (redshift, relatividad general)
delta_f_g = (GM / (c**2)) * (1 / r_earth - 1 / r_sat)  # fracción adimensional

# Neto (frecuencia relativa shift)
delta_f_net = delta_f_v + delta_f_g

# Desfase temporal neto por día (s)
delta_t_net = delta_f_net * t_day

# Error en distancia por día (km)
error_km_day = (c * abs(delta_t_net)) / 1000  # usamos absoluto ya que el neto es positivo (relojes satelitales adelantan)


print(f"Desfase neto: {delta_t_net * 1e6:.2f} μs/día")
print(f"Error acumulado sin corrección: {error_km_day:.2f} km/día")

Desfase neto: 38.44 μs/día
Error acumulado sin corrección: 11.52 km/día


### ***Corrección (Blue-Shift) Debido al Campo Gravitacional de la Tierra***

La métrica de Schwarzschild aparenta ser suficiente para corregir la dilatación debida al campo gravitacional de la Tierra [1,2]. Sin embargo, como bien sabemos, esta métrica considera que el sistema es simétrico esféricamente, lo cual no es del todo cierto, ya que:

1. La Tierra no es estática, está girando sobre su eje [5].
2. La Tierra no es una esfera perfecta [6].


Por lo tanto, para una descripción más precisa se utiliza la métrica de Kerr. Sin embargo, la corrección que se obtiene es del orden de $c^{-3}$, es decir, considerablemente más baja que los desfases producidos por los efectos principales.

La métrica de Minkowski (usando la convención $(+,-,-,-)$) es:

$$ ds^2 = \left(1 + \frac{2(\Phi - \Phi_0)}{c^2}\right) (cdt)^2 - \left(1 - \frac{2(\Phi - \Phi_0)}{c^2}\right) (dr^2 + r^2 d\theta^2 + r^2 \sin^2 \theta d\phi^2) $$

donde el potencial gravitacional debido a la Tierra está dado por:

$$ \Phi = -\frac{GM_E}{r} \left[1 - J_2 \left(\frac{a}{r}\right)^2 P_2(\cos \theta)\right] $$

y la constante gravitacional por la masa terrestre está dada por:

$$ GM_E = 3.986004418 \times 10^{14} \, \mathrm{m}^3 \mathrm{s}^{-2}, $$

el coeficiente del momento cuadrupolar de la Tierra es:

$$ J_2 = 1.0826300 \times 10^{-3}, $$

el radio ecuatorial de la Tierra:

$$ a = 6.3781370 \times 10^6 \, \mathrm{m}, $$

$P_2$ es el polinomio de Legendre de segundo grado y $\Phi_0$ hace referencia al potencial gravitacional sobre el geoide de la Tierra (una superficie equipotencial al nivel del mar sobre el marco de referencia ECEF, el cual describe a la Tierra como si rotara sobre un eje fijo (ver referencias para más información)).

Para mayor rigurosidad, recordemos que en la aproximación post-newtoniana de la relatividad general para campos débiles y velocidades bajas, la métrica se expande hasta orden $c^{-2}$, incorporando tanto el potencial newtoniano como correcciones relativistas. El término $J_2$ captura la oblatitud de la Tierra debido a su rotación, lo cual introduce una desviación del potencial esférico simétrico.

La velocidad en el marco inercial en coordenadas esféricas está dada por:

$$ v^2 = -\frac{dr^2 + r^2 d\theta^2 + r^2 \sin^2 \theta d\phi^2}{dt^2} $$

Reemplazando en (1), despreciando términos mayores a $c^{-2}$ y tomando la raíz cuadrada, se obtiene:

$$ ds = \left[1 + \frac{2(\Phi - \Phi_0)}{c^2} - \frac{v^2}{c^2}\right]^{1/2} (cdt) $$

Luego, usando que se cumple $d\tau = \frac{ds}{c}$, se obtiene:

$$ d\tau = \left[1 + \frac{2(\Phi - \Phi_0)}{c^2} - \frac{v^2}{c^2}\right]^{1/2} dt $$

La cual es la expresión del tiempo propio. Para aumentar la rigurosidad, notemos que esta derivación asume un observador en reposo relativo al marco rotante de la Tierra (es decir, $dr = d\theta = 0$, y $d\phi$ incluye el efecto co-rotante), y desprecia efectos de arrastre de marco que son de orden superior. Esta aproximación es válida para precisiones en GPS y relojes atómicos, como se detalla en las convenciones IERS.

In [15]:
#@title Código para el cálculo de la corrección por día

# Constantes de la imagen y valores estándar para GPS
GM = 3.986004418e14  # m³/s²
a = 6.378137e6       # Radio ecuatorial de la Tierra, m
r = 2.6561758e7      # Radio orbital aproximado para GPS (semi-eje mayor ~26,562 km), m
c = 2.99792458e8     # Velocidad de la luz, m/s
segundos_por_dia = 86400  # s/día

# Potenciales gravitacionales (aproximación esférica)
Phi0 = -GM / a
Phi_sat = -GM / r

# Velocidad orbital cuadrada
v2 = GM / r

# Tasa fraccional δ (reloj satélite avanza más rápido)
delta = (Phi_sat - Phi0) / c**2 - v2 / (2 * c**2)

# Ganancia de tiempo por día en segundos
delta_t = delta * segundos_por_dia

# Error en distancia por día en km (sin corrección, acumulado)
error_km = (c * delta_t) / 1000

print(f"Desfase neto: {delta_t * 1e6:.2f} μs/día")
print(f"Error acumulado sin corrección: {error_km:.2f} km/día")

Desfase neto: 38.44 μs/día
Error acumulado sin corrección: 11.52 km/día


### ***Efecto Sagnac en Relatividad Especial***

*Métrica en Coordenadas Cilíndricas Inerciales:*

Partimos de la métrica de Minkowski en coordenadas cilíndricas, que describe el espacio-tiempo plano en ausencia de rotación:

$$ ds^2 = c^2 dt^2 - dr^2 - r^2 d\phi^2 - dz^2 $$

*Transformación a Coordenadas Rotantes:*

Consideramos un sistema que rota con una velocidad angular uniforme $\omega_g$ respecto a un marco inercial. Realizamos una transformación de coordenadas para pasar al sistema rotante, donde el ángulo azimutal $\phi$ se transforma como $\phi' = \phi - \omega_g t$, mientras que $t = t'$, $r = r'$, y $z = z'$. Sustituyendo esta transformación en la métrica, obtenemos:

$$ ds^2 = c^2 dt'^2 - dr'^2 - r'^2 (d\phi' + \omega_g dt')^2 - dz'^2 $$

Expandiendo el término $r'^2 (d\phi' + \omega_g dt')^2$:

$$ r'^2 (d\phi' + \omega_g dt')^2 = r'^2 d\phi'^2 + 2 r'^2 \omega_g d\phi' dt' + r'^2 \omega_g^2 dt'^2 $$

Sustituyendo en la métrica:

$$ ds^2 = c^2 dt'^2 - dr'^2 - r'^2 d\phi'^2 - 2 r'^2 \omega_g d\phi' dt' - r'^2 \omega_g^2 dt'^2 - dz'^2 $$

Agrupando términos y definiendo $d\sigma^2 = -dr'^2 - r'^2 d\phi'^2 - dz'^2$ como el elemento espacial en el marco rotante:

$$ ds^2 = (c^2 - r'^2 \omega_g^2) dt'^2 - 2 r'^2 \omega_g d\phi' dt' + d\sigma^2 $$

*Geodésicas Nulas para Propagación de Luz:*

Para un rayo de luz, la geodésica es nula ($ds^2 = 0$). En un sistema rotante, asumimos que los términos de orden superior como $r'^2 \omega_g^2 dt'^2 / c^2$ son pequeños frente a $c^2 dt'^2$ (valido para $\omega_g r' \ll c$), por lo que podemos aproximar:

$$ ds^2 \approx c^2 dt'^2 - 2 r'^2 \omega_g d\phi' dt' + d\sigma^2 = 0 $$

Ignorando $d\sigma^2$ para trayectorias circulares en el plano ecuatorial ($dr' = dz' = 0$), tenemos:

$$ c^2 dt'^2 - 2 r'^2 \omega_g d\phi' dt' = 0 $$

Resolviendo para $dt'$:

$$ c^2 dt'^2 = 2 r'^2 \omega_g d\phi' dt' $$

$$ dt' \left( c^2 - \frac{2 r'^2 \omega_g d\phi'}{dt'} \right) = 0 $$

Dado que $dt' \neq 0$, se sigue que:

$$ c^2 = \frac{2 r'^2 \omega_g d\phi'}{dt'} $$

$$ dt' = \frac{2 r'^2 \omega_g d\phi'}{c^2} $$

*Tiempo de Viaje y Desfase:*

Integrando a lo largo de una trayectoria cerrada (e.g., un anillo de radio $r'$), donde $d\phi'$ varía de $0$ a $2\pi$ para un ciclo completo, y considerando que $r'^2 d\phi' / 2 = dA_e$ es un elemento diferencial de área proyectada en el plano ecuatorial, obtenemos:

$$ \Delta t' = \int \frac{2 r'^2 \omega_g d\phi'}{c^2} = \frac{2 \omega_g}{c^2} \int dA_e $$

Para un área total $A_e$ encerrada por la trayectoria:

$$ \Delta t' = \frac{4 \omega_g A_e}{c^2} $$

Este resultado indica el desfase temporal entre un rayo de luz que viaja en el sentido del movimiento rotacional y otro en sentido contrario, debido al efecto Sagnac. El factor 4 surge de la diferencia entre los tiempos de propagación en ambas direcciones.


In [14]:
#@title Código para el cálculo de la corrección por día

# Constantes
c = 299792458  # m/s
GM = 3.986e14  # m^3/s^2 (constante gravitacional Tierra)
r_sat = 2.656e7  # m (radio orbital GPS)
r_earth = 6.371e6  # m (radio Tierra)
t_day = 86400  # s (segundos en un día)

# Velocidad orbital del satélite
v = math.sqrt(GM / r_sat)  # m/s

# Efecto de velocidad (relatividad especial): delta_t_v en segundos por día
delta_f_v = - (v**2) / (2 * c**2)
delta_t_v = delta_f_v * t_day

# Efecto gravitacional (relatividad general): delta_t_g en segundos por día
delta_f_g = (GM / (c**2)) * (1 / r_earth - 1 / r_sat)
delta_t_g = delta_f_g * t_day

# Neto delta_t por día (s)
delta_t_net = delta_t_v + delta_t_g

# Error en distancia por día (km)
error_km_day = (c * delta_t_net) / 1000


print(f"Desfase Neto: {delta_t_net * 1e6:.2f} μs/día")
print(f"Error acumulado sin corrección: {error_km_day:.2f} km/día")

Desfase Neto: 38.50 μs/día
Error acumulado sin corrección: 11.54 km/día


## **Discusion de la aproximacion puramente relativista**


### **Proyecto SYPOR**

**SYPOR** (SYstèmes de POsitionnement Relativistes) es una propuesta/linea de trabajo para construir sistemas de posicionamiento por satélite **desde una base plenamente relativista**, en lugar de aplicar correcciones relativistas a un esquema esencialmente newtoniano (como hace hoy el GPS). La idea central es usar la propia estructura del espacio-tiempo y el tiempo propio de relojes a bordo para **definir coordenadas físicas (null-coordinates)** que permitan posicionar usuarios de forma autónoma y sin necesidad de sincronizaciones globales previas [7-9].

Es importante aclarar que SYPOR fue propuesto como alternativa conceptual para sistemas como Galileo — se plantea SYPOR como **un marco teórico y de diseño** (posicionamiento relativista y autolocalizado) más que como un programa espacial ya desplegado.

***Concepto operativo:***

Una constelación de satélites actúa como **cuatro (o más) relojes emisores** que transmiten continuamente su *tiempo propio*; las señales recibidas por un receptor definen directamente sus coordenadas espacio-temporales relativas a esa constelación (sistema de coordenadas nulas) [8]. En la versión **autolocalizada**, cada satélite no sólo emite su propio tiempo, sino también los tiempos que recibe de los demás; así la constelación se describe íntegramente en términos de los tiempos propios emitidos/intercambiados, sin necesidad de un marco terrestre para definir la métrica local [9].

***Ventajas propuestas frente al GPS clásico:***

**Marco coherente con la relatividad**: el sistema está formulado directamente en relatividad general, por lo que las “correcciones relativistas” dejan de ser parches y la posición/tiempo se obtiene de forma consistente con la métrica espacio-temporal.

**Autonomía de la constelación**: menor dependencia de estaciones en tierra para sincronización continua — potencialmente mayor resiliencia y nueva funcionalidad para estudios geodésicos y gravimetría relativista.  


***Desafíos técnicos y prácticos:***

**Relojes y enlaces**: requiere relojes muy estables (hoy los relojes atómicos ópticos ayudan) y enlaces inter-satélite fiables para intercambiar tiempos propios.

**Transformación a marcos terrestres**: para usos prácticos (mapas, navegación en la superficie) hace falta anclar o transformar las coordenadas nulas relativistas a sistemas terrestres; ese anclaje y las métricas locales introducen complejidad.   

**Implementación y transición**: cambiar de un GNSS clásico a uno diseñado como SYPOR implicaría rediseños de protocolos, receptores y operaciones de control de misión; en la práctica se estudian soluciones híbridas/compatibles.





## **Conclusiones**

Con este trabajo se demuestra la importancia del uso de la relatividad, tanto especial como general, en la precisión y exactitud de los GPS, demostrando que, sin estas correcciones, nuestros sistemas sufrirían desfases de decenas de kilómetros por día, lo que haría inservible el sistema rápidamente.

También se hizo un repaso sobre cómo la velocidad y el campo gravitatorio afectan el paso del tiempo, siendo el desfase de los satélites una prueba más de la corrección de las teorías de Einstein.

Además, se habló del proyecto SYPOR, una idea interesante, aunque todavía teórica, que busca un sistema de navegación satelital basado en las teorías de Einstein, en vez de la teoría newtoniana con correcciones que realmente son colocadas a mano, discutiendo sus ventajas pero también las dificultades para llevarlo a la práctica y sacarlo de la teoría.


## **Referencias**

- **[1]** Ashby, N. Relativity in the Global Positioning System. Living Rev. Relativ. 6, 1 (2003). https://doi.org/10.12942/lrr-2003-1
- **[2]** Parkinson, B. W., & Spilker, J. J. (Eds.). (1996). Progress in astronautics and aeronautics: Global positioning system: Theory and applications (Vol. 164). Aiaa.
- **[3]** Fukuyama, T., & Sugimoto, S. (2020). The Relativistic Corrections of GPS. arXiv preprint arXiv:2007.04582.
- **[4]** Zhang, J., Zhang, K., Grenfell, R., & Deakin, R. (2006). Short note: On the relativistic Doppler effect for precise velocity determination using GPS. Journal of Geodesy, 80(2), 104-110.
- **[5]** Rizzi, G., & Ruggiero, M. L. (Eds.). (2004). Relativity in rotating frames: relativistic physics in rotating reference frames. Dordrecht: Kluwer Academic Publishers.
- **[6]** Sośnica, K., Gałdyn, F. Orbital relativistic correction resulting from the Earth’s oblateness term. J Geod 99, 52 (2025). https://doi.org/10.1007/s00190-025-01973-3
- **[7]** Coll, B. (2013). Relativistic positioning systems: Perspectives and prospects. arXiv preprint arXiv:1302.5782.
- **[8]** Pascual‐Sánchez, J. F. (2007). Introducing relativity in global navigation satellite systems. Annalen der Physik, 519(4), 258-273.   
- **[9]** Delva, P., & Olympio, J. T. (2009). Mapping the spacetime metric with GNSS: a preliminary study. arXiv preprint arXiv:0912.4418.